# Lesson 3: Validation & Error Handling

In this lesson, you'll learn how to validate data and handle errors properly.

---

## How to Start These Lessons

If you're reading this in a text editor and need to open it in Jupyter:

```bash
cd /path/to/pyside_tutorial
python start_lessons.py
```

**To stop the Jupyter server:** Press `Ctrl+C` in the terminal.

---

## What You'll Learn

- Why validation matters
- How to check data types with `isinstance()`
- How to raise `ValueError` for bad input
- How to create custom exceptions
- The validation in our Movie and TVSeries classes

---

## The Problem: Bad Data

Remember our Movie class from Lesson 1? It accepts anything:

In [ ]:
# Movie without validation
class Movie:
    def __init__(self, title, year, director, duration):
        self.title = title
        self.year = year
        self.director = director
        self.duration = duration
    
    def __str__(self):
        return f"{self.title} ({self.year}) - {self.duration} min, {self.director}"

# All of these "work" but shouldn't:
bad1 = Movie("", 1999, "Director", 120)        # Empty title
bad2 = Movie("Title", 1800, "Director", 120)   # Movies didn't exist in 1800
bad3 = Movie("Title", 2020, "", 120)           # Empty director
bad4 = Movie("Title", 2020, "123", 120)        # Director is just numbers
bad5 = Movie("Title", 2020, "Director", -10)   # Negative duration

print(bad1)
print(bad2)

This is a problem because:
- Your app might crash later with confusing errors
- Bad data spreads through your system
- It's hard to find where the problem started

**Solution: Validate early, fail fast.** Check the data as soon as you receive it.

---

## Raising Errors with `raise`

The `raise` keyword lets you create an error on purpose:

In [ ]:
# Example of raising an error
def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero!")
    return a / b

print(divide(10, 2))  # Works fine
# print(divide(10, 0))  # Uncomment to see the error

Common error types:
- `ValueError` - The value is wrong (like a negative age)
- `TypeError` - The type is wrong (like passing a string when you need a number)

When you raise an error, the program stops and shows the error message. This is good! It tells you exactly what went wrong.

---

## Checking Types with `isinstance()`

Before checking values, we should check that we got the right type:

In [ ]:
# isinstance() checks if something is a certain type
print(isinstance("hello", str))     # True - "hello" is a string
print(isinstance(42, int))          # True - 42 is an integer
print(isinstance(3.14, float))      # True - 3.14 is a float
print(isinstance("hello", int))     # False - "hello" is not an integer

In [ ]:
# Using isinstance() for validation
def greet(name):
    if not isinstance(name, str):
        raise TypeError("name must be a string")
    print(f"Hello, {name}!")

greet("Alice")    # Works
# greet(123)      # Uncomment to see TypeError

---

## Adding Validation to Movie

Let's add validation to our Movie class. Here are the rules:

| Field | Rule |
|-------|------|
| title | Must be a non-empty string |
| year | Must be 1895 or later (first movie was made in 1895) |
| director | Must be a non-empty string with at least one letter |
| duration | Must be positive (greater than 0) |

In [ ]:
class Movie:
    def __init__(self, title, year, director, duration):
        # Type checks first
        if not isinstance(title, str):
            raise ValueError("Title must be a string")
        if not isinstance(director, str):
            raise ValueError("Director must be a string")
        
        # Value checks
        if not title.strip():  # strip() removes whitespace
            raise ValueError("Title cannot be empty")
        if year < 1895:
            raise ValueError("Year must be 1895 or later")
        if not director.strip():
            raise ValueError("Director cannot be empty")
        if not any(char.isalpha() for char in director):
            raise ValueError("Director must contain at least one letter")
        if duration <= 0:
            raise ValueError("Duration must be positive")
        
        # All checks passed - store the values
        self.title = title
        self.year = year
        self.director = director
        self.duration = duration
    
    def __str__(self):
        return f"{self.title} ({self.year}) - {self.duration} min, {self.director}"

In [ ]:
# Now bad data is rejected!
try:
    bad_movie = Movie("", 1999, "Director", 120)
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
# Try different bad inputs
test_cases = [
    ("", 1999, "Director", 120, "empty title"),
    ("Title", 1800, "Director", 120, "year too early"),
    ("Title", 2020, "123", 120, "director is just numbers"),
    ("Title", 2020, "Director", -10, "negative duration"),
]

for title, year, director, duration, description in test_cases:
    try:
        movie = Movie(title, year, director, duration)
        print(f"Created: {movie}")
    except ValueError as e:
        print(f"Rejected ({description}): {e}")

---

## Understanding the Validation Code

Let's break down some of the trickier parts:

### `not title.strip()`

`strip()` removes whitespace from the start and end of a string:

In [ ]:
print(repr("  hello  ".strip()))  # "hello"
print(repr("".strip()))            # ""
print(repr("   ".strip()))         # "" - only whitespace becomes empty

# Empty string is "falsy" in Python
print(bool(""))       # False
print(bool("hello"))  # True

# So 'not title.strip()' is True if title is empty or just whitespace
title = "   "
print(not title.strip())  # True - title is invalid

### `any(char.isalpha() for char in director)`

This checks if there's at least one letter in the director's name:

In [ ]:
# isalpha() checks if a character is a letter
print("a".isalpha())  # True
print("1".isalpha())  # False
print(" ".isalpha())  # False

# any() returns True if ANY item in the list is True
print(any([False, False, True]))   # True
print(any([False, False, False]))  # False

# Combine them to check for at least one letter
director1 = "Nolan"
director2 = "123"

print(any(char.isalpha() for char in director1))  # True - has letters
print(any(char.isalpha() for char in director2))  # False - no letters

---

## Custom Exceptions

Sometimes `ValueError` isn't specific enough. You can create your own exception types.

In our project, we have `MediaError` for errors specific to the media catalogue:

In [ ]:
# Custom exception from our project (media_catalogue.py lines 3-7)
class MediaError(Exception):
    """Custom exception for media-related errors."""
    def __init__(self, message, obj):
        super().__init__(message)
        self.obj = obj  # Store the problematic object

# Using it
try:
    bad_item = "not a movie"
    raise MediaError("Only Movie or TVSeries can be added", bad_item)
except MediaError as e:
    print(f"Error: {e}")
    print(f"Problem object: {e.obj}")
    print(f"Object type: {type(e.obj)}")

Why create a custom exception?

1. **Clarity** - `MediaError` is more descriptive than generic `Exception`
2. **Extra data** - We can store the problematic object (`e.obj`)
3. **Selective catching** - You can catch `MediaError` separately from other errors

---

## The Actual Movie Class with Validation

Here's the complete `Movie` class from our project (`media_catalogue/src/media_catalogue.py` lines 9-36):

In [ ]:
class Movie:
    def __init__(self, title, year, director, duration):
        """Initialize a Movie object with validation."""
        # Type checks
        if not isinstance(title, str):
            raise ValueError("Title must be a string")
        if not isinstance(director, str):
            raise ValueError("Director must be a string")

        # Value checks
        if not title.strip():
            raise ValueError("Title cannot be empty")
        if year < 1895:
            raise ValueError("Year must be 1895 or later")
        if not director.strip():
            raise ValueError("Director cannot be empty")
        if not any(char.isalpha() for char in director):
            raise ValueError("Director must contain at least one letter")
        if duration <= 0:
            raise ValueError("Duration must be positive")

        self.title = title
        self.year = year
        self.director = director
        self.duration = duration

    def __str__(self):
        return f'{self.title} ({self.year}) - {self.duration} min, {self.director}'

# Test valid movie
matrix = Movie("The Matrix", 1999, "The Wachowskis", 136)
print(matrix)

---

## TVSeries Validation

The `TVSeries` class inherits all of Movie's validation and adds its own:

In [ ]:
class TVSeries(Movie):
    def __init__(self, title, year, director, duration, seasons, total_episodes):
        # Call parent's __init__ - this does all the Movie validation
        super().__init__(title, year, director, duration)
        
        # Additional validation for TVSeries
        if seasons < 1:
            raise ValueError("Seasons must be 1 or greater")
        if total_episodes < 1:
            raise ValueError("Total episodes must be 1 or greater")
        
        self.seasons = seasons
        self.total_episodes = total_episodes
    
    def __str__(self):
        return f"{self.title} ({self.year}) - {self.seasons} seasons, {self.total_episodes} episodes, {self.duration} min avg, {self.director}"

# Valid series
bb = TVSeries("Breaking Bad", 2008, "Vince Gilligan", 47, 5, 62)
print(bb)

In [ ]:
# TVSeries inherits Movie's validation
try:
    bad_series = TVSeries("", 2008, "Creator", 30, 5, 100)  # Empty title
except ValueError as e:
    print(f"Caught from Movie validation: {e}")

try:
    bad_series = TVSeries("Show", 2008, "Creator", 30, 0, 100)  # 0 seasons
except ValueError as e:
    print(f"Caught from TVSeries validation: {e}")

Notice how `super().__init__()` does all the Movie validation automatically. That's the power of inheritance - we don't repeat the validation code.

---

## Try/Except for Handling Errors

When you call code that might raise an error, use `try/except` to handle it gracefully:

In [ ]:
def create_movie_safely(title, year, director, duration):
    """Try to create a movie, return None if it fails."""
    try:
        movie = Movie(title, year, director, duration)
        print(f"Created: {movie}")
        return movie
    except ValueError as e:
        print(f"Could not create movie: {e}")
        return None

# Try some movies
movie1 = create_movie_safely("Inception", 2010, "Nolan", 148)  # Works
movie2 = create_movie_safely("", 2010, "Nolan", 148)           # Fails gracefully

---

## Try It Yourself

### Exercise 1: Test the Validation

Try to create movies with different invalid inputs and see what errors you get:

In [ ]:
# Exercise 1: Try to create these movies and see what errors occur
# Uncomment each one to test

# movie1 = Movie(123, 2000, "Director", 90)  # Title is not a string
# movie2 = Movie("Title", 1890, "Director", 90)  # Year before 1895
# movie3 = Movie("   ", 2000, "Director", 90)  # Title is just whitespace
# movie4 = Movie("Title", 2000, "Director", 0)  # Duration is zero

### Exercise 2: Add Validation to a Class

Add validation to this `Product` class:
- `name` must be a non-empty string
- `price` must be positive (greater than 0)

In [ ]:
# Exercise 2: Add validation
class Product:
    def __init__(self, name, price):
        # TODO: Add validation here!
        # - name must be a non-empty string
        # - price must be positive (greater than 0)
        
        self.name = name
        self.price = price
    
    def __str__(self):
        return f"{self.name}: ${self.price:.2f}"

# Test cases - the first should work, the others should raise errors
try:
    p1 = Product("Widget", 9.99)
    print(f"Created: {p1}")
except ValueError as e:
    print(f"Error: {e}")

try:
    p2 = Product("", 9.99)  # Should fail - empty name
    print(f"Created: {p2}")  # This line runs if no validation
except ValueError as e:
    print(f"Correctly rejected: {e}")

try:
    p3 = Product("Widget", -5)  # Should fail - negative price
    print(f"Created: {p3}")  # This line runs if no validation
except ValueError as e:
    print(f"Correctly rejected: {e}")

<details>
<summary><b>Show Answer</b></summary>

```python
class Product:
    def __init__(self, name, price):
        if not isinstance(name, str):
            raise ValueError("Name must be a string")
        if not name.strip():
            raise ValueError("Name cannot be empty")
        if price <= 0:
            raise ValueError("Price must be positive")
        
        self.name = name
        self.price = price
```

</details>

### Exercise 3: Create a Custom Exception

Create a `ProductError` exception that stores both a message and the invalid product data:

In [ ]:
# Exercise 3: Create ProductError
class ProductError(Exception):
    def __init__(self, message, data):
        super().__init__(message)
        self.data = data  # TODO: Store the data!

# Test it
try:
    raise ProductError("Invalid product", {"name": "", "price": -5})
except ProductError as e:
    print(f"Error: {e}")
    print(f"Bad data: {e.data}")

<details>
<summary><b>Show Answer</b></summary>

```python
class ProductError(Exception):
    def __init__(self, message, data):
        super().__init__(message)
        self.data = data
```

</details>

---

## Key Takeaways

1. **Validate early** - Check data as soon as you receive it

2. **Use `isinstance()`** to check types before checking values

3. **Use `raise ValueError()`** when a value is wrong but the type is correct

4. **Custom exceptions** can store extra information about what went wrong

5. **Inherited validation** - When you use `super().__init__()`, you get the parent's validation automatically

6. **`try/except`** lets you handle errors gracefully instead of crashing

---

## Next Lesson

You'll learn:
- How to build the `MediaCatalogue` class
- Working with lists of objects
- Filtering with list comprehensions
- The `add()`, `delete()`, and filter methods

---

## Navigation

| | |
|:---|---:|
| [**← Previous: Lesson 2 - Inheritance**](02_inheritance.ipynb) | [**Next: Lesson 4 - Collections →**](04_collections.ipynb) |